# Tabular RAG with LM Studio
This notebook demonstrates how to use LM Studio for a tabular Retrieval-Augmented Generation (TRAG)

In [17]:
!pip install dotenv openai sqlalchemy "pandas==2.3.1" "tabulate==0.9.0"

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [18]:
from utils.llm_client import lm_studio_client, LM_STUDIO_MODEL
from sqlalchemy import create_engine
import pandas as pd
import json

Can LLM response without any context?
Let's try to ask a question without any context.

In [19]:
user_query = '''What's the release year of 'X'?'''

prompt = f"""
User Query: {user_query}
"""

response = lm_studio_client.chat.completions.create(
    model=LM_STUDIO_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

response_text = response.choices[0].message.content

print(response_text)

I’m not sure which “X” you’re referring to. Could you let me know the full title or provide a bit more context? Then I can give you the release year.


Now we need to generate a SQL query based on the user query.
We will use LM Studio to generate the SQL query.

In [20]:
user_query = '''What's the release year of 'X'?'''

prompt = f"""
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
"""

response = lm_studio_client.chat.completions.create(
    model=LM_STUDIO_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

response_text = response.choices[0].message.content

print(response_text)

```sql
-- Retrieve the release year for a specific movie title.
SELECT release_year
FROM movies
WHERE title = 'X';
```
This query assumes the movie data is stored in a table named **movies** with a column **release_year** that holds the year the film was released. Replace `'X'` with the actual title of the movie you’re interested in.


Well it generated a SQL query, but we need to execute it against the database to get the answer.
Let's assume we have a database connection set up and we can execute the query.

In [21]:
engine = create_engine(f'sqlite:///datasets/movies.db')

movie_name = pd.read_sql('select title from movies', engine).sample(1).title.values[0]

print('Working with Movie Name: ', movie_name )

Working with Movie Name:  Linguistics Gaming Of The Age Driving


# Now we can execute the SQL query generated by the LLM against the database.

In [22]:
# Sample Queries
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///datasets/movies.db
"""

user_queries = [
    f"What's the release year of '{movie_name}'?",
    f"Who directed the movie '{movie_name}'?",
    f"List all actors in the movie '{movie_name}'.",
    f"What are the genres of the movie '{movie_name}'?",
]
sql_queries = []
for user_query in user_queries:
    response = lm_studio_client.chat.completions.create(
        model=LM_STUDIO_MODEL,
        messages=[
            {"role": "user", "content": prompt.format(user_query=user_query)}
        ]
    )

    resp = response.choices[0].message.content
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    sql_queries.append(sql)
    print(user_query, ':', sql)
    print('-' * 80)
    print()

What's the release year of 'Linguistics Gaming Of The Age Driving'? : SELECT release_year
FROM movies
WHERE title = 'Linguistics Gaming Of The Age Driving';
--------------------------------------------------------------------------------

Who directed the movie 'Linguistics Gaming Of The Age Driving'? : -- Retrieve the director(s) of a movie named "Linguistics Gaming Of The Age Driving"

SELECT d.name AS director_name
FROM movies m
JOIN movie_directors md ON m.id = md.movie_id          -- link movie to its directors
JOIN directors d ON md.director_id = d.id              -- get director details
WHERE m.title = 'Linguistics Gaming Of The Age Driving';
--------------------------------------------------------------------------------

List all actors in the movie 'Linguistics Gaming Of The Age Driving'. : SELECT a.name
FROM actors AS a
JOIN cast_members   AS cm ON a.id = cm.actor_id
JOIN movies         AS m  ON cm.movie_id = m.id
WHERE m.title = 'Linguistics Gaming Of The Age Driving';
----

Let's execute the SQL queries against the database and get the results.

In [23]:
for sql in sql_queries:
    try:
        print(pd.read_sql(sql, engine).to_markdown(index=False))
    except Exception as e:
        print(f"Error executing query: {e}")
    print('-' * 80)
    print()

|   release_year |
|---------------:|
|           2020 |
--------------------------------------------------------------------------------

Error executing query: (sqlite3.OperationalError) no such table: movie_directors
[SQL: -- Retrieve the director(s) of a movie named "Linguistics Gaming Of The Age Driving"

SELECT d.name AS director_name
FROM movies m
JOIN movie_directors md ON m.id = md.movie_id          -- link movie to its directors
JOIN directors d ON md.director_id = d.id              -- get director details
WHERE m.title = 'Linguistics Gaming Of The Age Driving';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
--------------------------------------------------------------------------------

Error executing query: (sqlite3.OperationalError) no such column: a.name
[SQL: SELECT a.name
FROM actors AS a
JOIN cast_members   AS cm ON a.id = cm.actor_id
JOIN movies         AS m  ON cm.movie_id = m.id
WHERE m.title = 'Linguistics Gaming Of The Age Driving';]
(Background o

Lets Retry with schema details/data dictionary

In [24]:
database_schema = '''

#### 1\. `movies` Table

  * **Description:** Stores core information about each movie.
  * **Columns:**
      * `movie_id` (INTEGER, Primary Key): Unique identifier for the movie.
      * `title` (TEXT, NOT NULL): The title of the movie.
      * `release_year` (INTEGER): The year the movie was released.
      * `duration_minutes` (INTEGER): The length of the movie in minutes.
      * `synopsis` (TEXT): A brief summary of the movie's plot.
      * `budget_usd` (REAL): The production budget of the movie in USD.
      * `box_office_gross_usd` (REAL): The worldwide box office revenue in USD.
      * `director_id` (INTEGER, Foreign Key to `directors.director_id`): The director of the movie.
      * `studio_id` (INTEGER, Foreign Key to `studios.studio_id`): The studio that produced the movie.

#### 2\. `actors` Table

  * **Description:** Stores information about individual actors.
  * **Columns:**
      * `actor_id` (INTEGER, Primary Key): Unique identifier for the actor.
      * `actor_name` (TEXT, NOT NULL): The full name of the actor.
      * `birth_date` (TEXT): The actor's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The actor's nationality.

#### 3\. `directors` Table

  * **Description:** Stores information about movie directors.
  * **Columns:**
      * `director_id` (INTEGER, Primary Key): Unique identifier for the director.
      * `director_name` (TEXT, NOT NULL): The full name of the director.
      * `birth_date` (TEXT): The director's birth date (YYYY-MM-DD format).
      * `nationality` (TEXT): The director's nationality.

#### 4\. `studios` Table

  * **Description:** Stores information about movie production studios.
  * **Columns:**
      * `studio_id` (INTEGER, Primary Key): Unique identifier for the studio.
      * `studio_name` (TEXT, NOT NULL): The name of the studio.
      * `founded_year` (INTEGER): The year the studio was founded.
      * `country` (TEXT): The country where the studio is primarily based.

#### 5\. `genres` Table

  * **Description:** Stores a list of movie genres.
  * **Columns:**
      * `genre_id` (INTEGER, Primary Key): Unique identifier for the genre.
      * `genre_name` (TEXT, NOT NULL, UNIQUE): The name of the genre (e.g., "Action", "Drama", "Sci-Fi").

#### 6\. `cast_members` Table (Junction Table)

  * **Description:** Links movies to actors, specifying the role an actor played in a particular movie. This handles the many-to-many relationship between `movies` and `actors`.
  * **Columns:**
      * `cast_id` (INTEGER, Primary Key): Unique identifier for a cast entry.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `actor_id` (INTEGER, Foreign Key to `actors.actor_id`): The actor ID.
      * `role_name` (TEXT): The name of the character played by the actor in that movie.

#### 7\. `movie_genres` Table (Junction Table)

  * **Description:** Links movies to genres, allowing a movie to have multiple genres. This handles the many-to-many relationship between `movies` and `genres`.
  * **Columns:**
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie ID.
      * `genre_id` (INTEGER, Foreign Key to `genres.genre_id`): The genre ID.
      * (Composite Primary Key: `movie_id`, `genre_id` to ensure unique genre assignments per movie).

#### 8\. `reviews` Table

  * **Description:** Stores user reviews and ratings for movies.
  * **Columns:**
      * `review_id` (INTEGER, Primary Key): Unique identifier for the review.
      * `movie_id` (INTEGER, Foreign Key to `movies.movie_id`): The movie being reviewed.
      * `reviewer_name` (TEXT): The name or alias of the reviewer.
      * `rating` (INTEGER): The numerical rating given (e.g., 1-10).
      * `review_text` (TEXT): The full text of the review.
      * `review_date` (TEXT): The date the review was posted (YYYY-MM-DD format).

'''

# Sample Queries
prompt = """
You are an AI assistant that can answer questions about a movie database.
The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
Your task is to answer the user's query based on the provided database information.

User Query: {user_query}

Generate a SQL query to retrieve the requested information from the database.
database: sqlite:///movies.db
schema details:
{database_schema}
"""

sql_queries = []
for user_query in user_queries:
    response = lm_studio_client.chat.completions.create(
        model=LM_STUDIO_MODEL,
        messages=[
            {"role": "user", "content": prompt.format(user_query=user_query, database_schema=database_schema)}
        ]
    )

    resp = response.choices[0].message.content
    sql = resp.strip().split('```sql')[1].split(
        '```')[0].strip()  # Extract SQL query from response
    sql_queries.append(sql)
    print(user_query, ':', sql)
    print('-' * 80)
    print()

    for sql in sql_queries:
        try:
            print(pd.read_sql(sql, engine).to_markdown(index=False))
        except Exception as e:
            print(f"Error executing query: {e}")
        print('-' * 80)
        print()

What's the release year of 'Linguistics Gaming Of The Age Driving'? : SELECT release_year
FROM movies
WHERE title = 'Linguistics Gaming Of The Age Driving';
--------------------------------------------------------------------------------

|   release_year |
|---------------:|
|           2020 |
--------------------------------------------------------------------------------

Who directed the movie 'Linguistics Gaming Of The Age Driving'? : SELECT d.director_name
FROM movies m
JOIN directors d ON m.director_id = d.director_id
WHERE m.title = 'Linguistics Gaming Of The Age Driving';
--------------------------------------------------------------------------------

|   release_year |
|---------------:|
|           2020 |
--------------------------------------------------------------------------------

| director_name   |
|:----------------|
| animato-chain   |
--------------------------------------------------------------------------------

List all actors in the movie 'Linguistics Gaming 

---
# Let's employ LLM to do everything

In [25]:
def get_schema(table_name, engine=engine):
    """
    Retrieves the schema of a specified table in the SQLite database.

    Args:
        table_name (str): The name of the table to retrieve schema for.
        engine: SQLAlchemy engine connected to the SQLite database.

    Returns:
        str: Formatted string representing the table schema.
    """
    conn = engine.connect()
    columns_info = conn.exec_driver_sql(
        f'PRAGMA table_info("{table_name}")').fetchall()

    table_columns = [(col[1], col[2]) for col in columns_info]

    formatted_string = f"{table_name}:\n"
    for col_name, col_type in table_columns:
        formatted_string += f" {col_name} ({col_type})\n"

    return formatted_string


def discover_tables(user_query, engine=engine):

  tables = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';", engine).to_markdown(index=False)

  table_choice_prompt = f'''

  You are an experienced Engineer and have been given the task of being the data retriever.

  Given the user query, make a choice on the tables to be used to fetch the data and then be able to build the answer.

  user query: {user_query}

  table names: {tables}

  Respond with the table names to be used as a json list of table names
  '''

  
  response = lm_studio_client.chat.completions.create(
    model=LM_STUDIO_MODEL,
    messages=[
        {"role": "user", "content": table_choice_prompt}
    ]
  )

  table_choice = response.choices[0].message.content
  json_parsed = json.loads(table_choice.split('```json')[-1].split('```')[0])
  return (json_parsed)


def build_ddl_prompt(json_parsed, engine=engine):
  schemas = []

  for table_name in json_parsed:
    schemas.append(get_schema(table_name, engine=engine))

  schemas_prompt = '\n\n'.join(schemas)

  return (schemas_prompt)


def r_of_rag(user_query, schemas_prompt, engine):

  prompt = """
  You are an AI assistant that can answer questions about a movie database.
  The database contains information about movies, directors, actors, studios, genres, cast members, and reviews.
  Your task is to answer the user's query based on the provided database information.

  User Query: {user_query}

  Generate a SQL query to retrieve the requested information from the database.
  database: sqlite:///movies.db

  database schema:
  {ddl}

  """
  response = lm_studio_client.chat.completions.create(
    model=LM_STUDIO_MODEL,
    messages=[
        {"role": "user", "content": prompt.format(user_query=user_query, ddl=schemas_prompt)}
    ]
  )

  resp = response.choices[0].message.content
  try:

    sql = resp.strip().split('```sql')[1].split(
      '```')[0].strip()  # Extract SQL query from response

    print('-'*80)
    print(sql)
    print('-'*80)
    data_response = pd.read_sql(sql, engine).to_markdown(index=False)
    return data_response
  except Exception as e:
    print(e)
  return ""


def ag_of_rag(user_query, data_response, engine, chat_history):
  rag_prompt = '''
  You are an AI assistant that can answer questions about a movie database.
  The database contains information about movies, directors, actors, studios, genres, cast members,
  and reviews.
  Your task is to answer the user's query based on the provided database information.

  User Query: {user_query}

  Chat History:

  {chat_history}

  Database Response:
  {data_response}

  '''

  try:
      response = lm_studio_client.chat.completions.create(
        model=LM_STUDIO_MODEL,
        messages=[
            {"role": "user", "content": rag_prompt.format(
                user_query=user_query,
                data_response=data_response,
                chat_history=chat_history
            )}
        ]
      )

      llm_resp = response.choices[0].message.content
      

      return llm_resp

  except Exception as e:
      print(f"Error executing query: {e}")

### Let's see the result.

In [26]:
user_queries = [
    f'I need to understand the popularity of the movie {movie_name}',
    f'What are the movies in the similar popularity scale or higher?',
    f'What are the most common Genres for these movies?',
    f'Ok, I want to undertand what could be a common cast to such successful movies. Rank them by their performance in terms of overall movie rating'
]

history = []

for user_query in user_queries[:]:
  json_parsed = discover_tables(user_query)
  schemas_prompt = build_ddl_prompt(json_parsed)
  data_response = r_of_rag(user_query, schemas_prompt, engine)
  # print(data_response)
  history.append(f'data_context: \n{data_response}')
  resp = ag_of_rag(user_query, data_response, engine, history)
  history.append(f'user_query: {user_query}\nanswer: {resp}')
  print(user_query, ':', resp)
  print('-'*80)

--------------------------------------------------------------------------------
-- Popularity of “Linguistics Gaming Of The Age Driving”
SELECT 
    m.title,
    COUNT(r.review_id)          AS review_count,          -- how many reviews
    ROUND(AVG(r.rating), 2)     AS avg_rating,            -- average rating (0–10 or whatever scale is used)
    SUM(CASE WHEN r.rating >= 8 THEN 1 ELSE 0 END) AS high_rating_count, -- # of top‑rated reviews
    MIN(r.review_date)         AS first_review_date,
    MAX(r.review_date)         AS last_review_date
FROM movies m
JOIN reviews r ON m.movie_id = r.movie_id
WHERE m.title = 'Linguistics Gaming Of The Age Driving'
GROUP BY m.movie_id, m.title;
--------------------------------------------------------------------------------
I need to understand the popularity of the movie Linguistics Gaming Of The Age Driving : **Popularity Snapshot – “Linguistics Gaming Of The Age Driving”**

| Metric | Value | What It Means |
|--------|-------|---------------|
| 